In [1]:
import datetime
import calendar

from pycds import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_, or_
from sqlalchemy.sql import func
from sqlalchemy import funcfilter
from sqlalchemy import within_group
from sqlalchemy import select

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pgpasslib

In [3]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind = engine)
session = Session()

In [4]:
year = 2000 
month = 7
day = 1
interval = 1
yr_interval = 10

start_year = datetime.datetime(year, month, day)
end_year = datetime.datetime(year + yr_interval, month, day)

start_time = datetime.datetime(year, month, day)
end_time = datetime.datetime(year, month + interval, day)

In [29]:
def baseline(start_time, end_time):
    query = session.query(Obs.datum, Obs.time, Variable.standard_name, History.station_id).filter(Obs.time <= end_time).filter(Obs.time >= start_time).join(History).join(Variable)
    return query

def precip_query(start_time, end_time):
    query = baseline(start_time, end_time).filter(Variable.standard_name == "lwe_thickness_of_precipitation_amount").filter(or_(Variable.description == "Total precipiation", Variable.id == 1397))
    return query

def temp_query(start_time, end_time):
    query = baseline(start_time, end_time).filter(Variable.standard_name == "air_temperature").filter(or_(Variable.description == "Air temperature", Variable.id == 1510))
    return query

def annual_rain(start_year, end_year, mean = True):
    yr_interval = float(np.abs(end_year.year - start_year.year))
    print(yr_interval)
    query = session.query(func.sum(Obs.datum*0.1/yr_interval).label("sum"), func.max(Obs.time).label("max_date"), History.lat, History.lon, History.station_id).group_by(History).having(func.max(Obs.time) >= end_year).filter(Variable.standard_name == "lwe_thickness_of_precipitation_amount").filter(or_(Variable.description == "Total precipiation", Variable.id == 1397))
    query = query.filter(Obs.time <= end_year).filter(Obs.time >= start_year).join(History).join(Variable)
    return query

def design_temp_25(start_time, end_time, month = 7):
    query = session.query(Obs.datum, 
                          Obs.time,
                          Obs.history) \
    .filter(Variable.standard_name == "air_temperature") \
    .filter(or_(Variable.description == "Air temperature", Variable.id == 1510))
    query = query.filter(func.extract("month", Obs.time) == month)
    query = query.filter(func.extract("year", Obs.time) <= end_time.year).filter(func.extract("year", Obs.time) >= start_time.year).join(History).join(Variable)
    #percentiles = stations.get(func.percentile_disc(0.025).within_group(Obs.datum).label('percentile')).label('percentiles')

   # query = select([stations.Obs.time, percentiles.percentile]).order_by(stations.Obs.time)
    
    return query

def show_design_values():
    query = session.query(Variable.standard_name, Variable.description, Variable.unit, Variable.id)
    return query


In [30]:
q = design_temp_25(start_time, end_time)
#q = show_design_values()
print(q)

SELECT crmp.obs_raw.datum AS crmp_obs_raw_datum, crmp.obs_raw.obs_time AS crmp_obs_raw_obs_time, crmp.meta_history.history_id = crmp.obs_raw.history_id AS history 
FROM crmp.obs_raw JOIN crmp.meta_history ON crmp.meta_history.history_id = crmp.obs_raw.history_id JOIN crmp.meta_vars ON crmp.meta_vars.vars_id = crmp.obs_raw.vars_id 
WHERE crmp.meta_vars.standard_name = %(standard_name_1)s AND (crmp.meta_vars.long_description = %(long_description_1)s OR crmp.meta_vars.vars_id = %(vars_id_1)s) AND EXTRACT(month FROM crmp.obs_raw.obs_time) = %(param_1)s AND EXTRACT(year FROM crmp.obs_raw.obs_time) <= %(param_2)s AND EXTRACT(year FROM crmp.obs_raw.obs_time) >= %(param_3)s


In [31]:
df = pd.read_sql(q.statement, engine)

In [32]:
df

,datum,obs_time,history
0,155.0,2000-07-01,True
1,158.0,2000-07-01,True
2,130.0,2000-07-02,True
3,155.0,2000-07-03,True
4,148.0,2000-07-04,True
5,130.0,2000-07-05,True
6,155.0,2000-07-06,True
7,162.0,2000-07-07,True
8,180.0,2000-07-08,True
9,158.0,2000-07-09,True


In [ ]:
sns.distplot(df['datum'])
plt.xlabel('July Temperature [0.1 C]')
plt.ylabel('KDE Frequency')
plt.savefig('temp_percentile_25')

In [9]:
plt.scatter(df['lon'], df['lat'])
plt.ylim(45, 60)

KeyError: 'lon'